# Only using sensors data

## Config env and callback function

In [ ]:
# ENV CONF
import carla 
# client = carla.Client("localhost", 2000)
# world = client.load_world('Town01')
import gymnasium as gym
from carla_env import CarEnv
import numpy as np
import time
import matplotlib.pyplot as plt
from stable_baselines3.common.callbacks import BaseCallback
class CARLA_G(gym.Env):
    def __init__(self, ):
        super(CARLA_G, self).__init__()
        self.env = CarEnv()
        self.action_space = gym.spaces.Box(low=-1, high=1, shape = (2, ), dtype=np.float32)
        self.observation_space = gym.spaces.Box(low = -np.inf, high = np.inf, shape=(16, ), dtype=np.float32)

    def step(self, action):
        [new_image, new_state], reward, done, info = self.env.step(action)
        return new_state.astype(np.float32), reward, done, False, {}
    
    def reset(self, seed = None, options = {}):
        image, state = self.env.reset()
        return state.astype(np.float32), {}
    
    def render(self):
        pass


class MyCallback(BaseCallback):
    def __init__(self, verbose=0):
        super(MyCallback, self).__init__(verbose)
        self.episode_rewards = []
        self.episode_lengths = []

    def _on_step(self) -> bool:
        # Check if there are completed episodes
        if len(self.model.ep_info_buffer) > 0:
            info = self.model.ep_info_buffer[-1]
            self.episode_rewards.append(info['r'])
            self.episode_lengths.append(info['l'])

            # Here we log the episodic rewards and lengths to TensorBoard
            self.logger.record('episode_reward', np.mean(self.episode_rewards[-100:]))
            self.logger.record('episode_length', np.mean(self.episode_lengths[-100:]))
            self.logger.dump(step=self.num_timesteps)

        return True


env = CARLA_G()
town_name = 'Town01'
env.env.town_name = town_name
env.reset()

In [ ]:
from stable_baselines3 import SAC, PPO, DDPG
from stable_baselines3.common.env_checker import check_env
import os
from stable_baselines3.common.logger import configure
log_dir = "./logs"
os.makedirs(log_dir, exist_ok=True)

#model conf
#---new model
model = SAC("MlpPolicy", env, verbose=1,buffer_size=1000000,tensorboard_log=log_dir)

#---load model
# if you countinue learning from saved model, uncomment this lines:
# model = SAC.load("model name", print_system_info=True)
# model2 = SAC("MlpPolicy", env, verbose=1)
# model.env = model2.env
# model.load_replay_buffer("./buffer_SAC_model_sesnor.pkl")



# Pass a TensorBoard logger to the model
new_logger = configure(log_dir, ["stdout", "tensorboard"])


# Instantiate the callback
my_callback = MyCallback()
model.set_logger(new_logger)

for i in range(10000000):
    try:
        name = "SAC_model_sesnor_run_"+str(i+1)
        print(name)
        model.learn(total_timesteps=5000, log_interval=4,callback = my_callback)
        model.save(name)
        #saving realy buffer for next learning process
        model.save_replay_buffer("buffer_SAC_model_sesnor.pkl")
    except:
        print("error")
        time.sleep(0.2)